# 03 - End-to-End Trip Planning Demo

**Purpose:** This notebook demonstrates the complete multi-agent workflow from user input to final itinerary generation.

**Workflow Steps:**
1. Initialize the LangGraph multi-agent system
2. Execute flight search
3. Execute hotel search
4. Fetch weather data
5. Search for attractions and restaurants
6. Generate comprehensive itinerary
7. Calculate budget breakdown

**Expected Outcome:** A complete, markdown-formatted travel plan ready for user consumption.

## Setup and Imports

In [ ]:
import os
import sys
import json
import uuid
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

# Load environment
load_dotenv()

# Add src to path
sys.path.append('../src')

from src.agent.agentic_workflow import GraphBuilder

print("✅ Environment loaded")
print(f"📅 Demo Date: {datetime.now().strftime('%Y-%m-%d')}")

## 1. Define Trip Parameters

In [ ]:
# User inputs
trip_params = {
    "from_city": "Mumbai",
    "destination": "Goa",
    "start_date": (datetime.now() + timedelta(days=14)).strftime("%Y-%m-%d"),
    "days": 4,
    "travelers": 2,
    "budget": "Moderate",
    "vibe": "Relaxed",
    "special_requests": "Vegetarian food preferences, beach activities"
}

# Calculate checkout date
checkout_date = (datetime.strptime(trip_params["start_date"], "%Y-%m-%d") + 
                 timedelta(days=trip_params["days"])).strftime("%Y-%m-%d")

print("🎯 Trip Configuration")
print("="*80)
for key, value in trip_params.items():
    print(f"{key.replace('_', ' ').title()}: {value}")
print(f"Checkout Date: {checkout_date}")

## 2. Initialize Multi-Agent System

In [ ]:
print("🚀 Initializing Multi-Agent System...")
print("="*80)

# Build the graph
graph = GraphBuilder(model_provider="groq")()

print("\n✅ System ready")
print("   - LangGraph compiled")
print("   - All tools registered")
print("   - Agents initialized")

## 3. Construct Agent Prompt

In [ ]:
# Build comprehensive prompt
agent_prompt = f"""
TRIP PLANNING REQUEST

📋 **TRIP PARAMETERS:**
- Origin: {trip_params['from_city']}
- Destination: {trip_params['destination']}
- Start Date: {trip_params['start_date']}
- End Date: {checkout_date}
- Duration: {trip_params['days']} days
- Travelers: {trip_params['travelers']} people
- Budget Level: {trip_params['budget']}
- Trip Vibe: {trip_params['vibe']}
- Special Requests: {trip_params['special_requests']}

🤖 **MULTI-AGENT EXECUTION PROTOCOL:**

**STEP 1 - Flight Agent:**
Execute: search_flights(origin="{trip_params['from_city']}", destination="{trip_params['destination']}", travel_date="{trip_params['start_date']}")
→ Filter by price, layovers, travel time
→ Display ALL flights in Budget/Moderate/Premium categories

**STEP 2 - Hotel Agent:**
Execute: search_hotels(location="{trip_params['destination']}", check_in_date="{trip_params['start_date']}", check_out_date="{checkout_date}")
→ Analyze by location, budget, amenities
→ Display ALL hotels in Budget/Moderate/Luxury categories

**STEP 3 - Weather Forecast:**
Execute: get_weather_forecast(city="{trip_params['destination']}", travel_date="{trip_params['start_date']}")

**STEP 4 - Place Discovery:**
Execute: search_attractions(place="{trip_params['destination']}")
Execute: search_restaurants(place="{trip_params['destination']}")
Execute: search_activities(place="{trip_params['destination']}")

**STEP 5 - Generate Complete Itinerary:**
Create a detailed {trip_params['days']}-day itinerary with:
- Day-by-day schedule with REAL attraction names
- Specific restaurants for each meal
- Activities matching the "{trip_params['vibe']}" vibe
- Cost estimates in INR (₹)
- Complete budget breakdown

Execute this multi-agent workflow now and provide the complete markdown itinerary.
"""

print("📝 Agent Prompt Constructed")
print("="*80)
print(agent_prompt[:500] + "...")
print("\n[Full prompt prepared for agent execution]")

## 4. Execute Multi-Agent Workflow

In [ ]:
import time

print("⚙️ Executing Multi-Agent Workflow...")
print("="*80)

# Create initial state
initial_state = {
    "messages": [HumanMessage(content=agent_prompt)],
    "tool_calls_count": 0
}

# Generate unique thread ID
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

print(f"Thread ID: {thread_id}\n")

# Track execution time
start_time = time.time()

# Execute the graph
try:
    final_state = graph.invoke(initial_state, config=config)
    execution_time = time.time() - start_time
    
    print("\n" + "="*80)
    print(f"✅ Workflow completed in {execution_time:.2f} seconds")
    print(f"   Total tool calls: {final_state.get('tool_calls_count', 'N/A')}")
    print(f"   Messages exchanged: {len(final_state['messages'])}")
    
except Exception as e:
    print(f"\n❌ Workflow failed: {str(e)}")
    raise

## 5. Extract and Display Results

In [ ]:
# Extract final response
last_message = final_state["messages"][-1]
content = last_message.content

# Handle different content formats
if isinstance(content, list):
    final_itinerary = "".join(
        c.get("text", "") for c in content if isinstance(c, dict)
    )
else:
    final_itinerary = str(content)

print("📄 Generated Travel Plan")
print("="*80)
print(final_itinerary)

## 6. Analyze Tool Usage

In [ ]:
# Analyze which tools were called
import re
from collections import Counter

tool_calls = []
for msg in final_state["messages"]:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            if isinstance(tc, dict):
                tool_calls.append(tc.get('name', 'unknown'))

if tool_calls:
    tool_counter = Counter(tool_calls)
    
    print("📊 Tool Usage Statistics")
    print("="*80)
    for tool, count in tool_counter.most_common():
        print(f"   {tool}: {count} call(s)")
    
    # Visualize
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    tools = list(tool_counter.keys())
    counts = list(tool_counter.values())
    
    plt.barh(tools, counts, color='skyblue', alpha=0.7)
    plt.xlabel('Number of Calls', fontsize=12)
    plt.ylabel('Tool Name', fontsize=12)
    plt.title('Multi-Agent Tool Usage Distribution', fontsize=14)
    plt.grid(True, axis='x', alpha=0.3)
    
    for i, v in enumerate(counts):
        plt.text(v + 0.1, i, str(v), va='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No tool usage data available")

## 7. Save Travel Plan

In [ ]:
# Save to markdown file
output_dir = "../outputs"
os.makedirs(output_dir, exist_ok=True)

filename = f"trip_plan_{trip_params['from_city']}_to_{trip_params['destination']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
filepath = os.path.join(output_dir, filename)

with open(filepath, 'w', encoding='utf-8') as f:
    f.write(final_itinerary)

print(f"💾 Travel plan saved to: {filepath}")
print(f"   File size: {len(final_itinerary):,} characters")

## 8. Extract Key Metrics from Itinerary

In [ ]:
# Extract budget information
import re

print("💰 Budget Analysis")
print("="*80)

# Try to find grand total
grand_total_match = re.search(r'GRAND TOTAL.*?₹([\d,]+)', final_itinerary, re.IGNORECASE)
if grand_total_match:
    total = grand_total_match.group(1)
    print(f"\nTotal Trip Cost: ₹{total}")
    
    # Calculate per person
    try:
        total_num = int(total.replace(',', ''))
        per_person = total_num / trip_params['travelers']
        print(f"Cost Per Person: ₹{per_person:,.2f}")
    except:
        pass

# Extract flight info
flight_match = re.search(r'Flight.*?₹([\d,]+)', final_itinerary, re.IGNORECASE)
if flight_match:
    print(f"\nFlight Cost: ₹{flight_match.group(1)}")

# Extract hotel info
hotel_match = re.search(r'Hotel.*?₹([\d,]+)', final_itinerary, re.IGNORECASE)
if hotel_match:
    print(f"Accommodation Cost: ₹{hotel_match.group(1)}")

print("\n✅ Financial breakdown extracted from itinerary")

## 9. Validate Itinerary Completeness

In [ ]:
# Check if all required sections are present
required_sections = [
    "Flight Options",
    "Hotels",
    "Weather",
    "Itinerary",
    "Budget"
]

print("✔️ Itinerary Completeness Check")
print("="*80)

for section in required_sections:
    if section.lower() in final_itinerary.lower():
        print(f"✅ {section} section found")
    else:
        print(f"❌ {section} section MISSING")

# Check for day-by-day breakdown
day_count = len(re.findall(r'\*\*Day \d+:', final_itinerary, re.IGNORECASE))
print(f"\n📅 Day-by-day entries found: {day_count}")

if day_count == trip_params['days']:
    print(f"✅ All {trip_params['days']} days covered")
else:
    print(f"⚠️ Expected {trip_params['days']} days, found {day_count}")

# Word count
word_count = len(final_itinerary.split())
print(f"\n📝 Itinerary statistics:")
print(f"   Total words: {word_count:,}")
print(f"   Total characters: {len(final_itinerary):,}")
print(f"   Lines: {len(final_itinerary.split(chr(10))):,}")

## 10. Test Different Trip Scenarios

In [ ]:
# Define multiple test scenarios
test_scenarios = [
    {
        "name": "Budget Weekend Getaway",
        "params": {
            "from_city": "Delhi",
            "destination": "Jaipur",
            "days": 2,
            "budget": "Cheap",
            "vibe": "Cultural"
        }
    },
    {
        "name": "Luxury Beach Vacation",
        "params": {
            "from_city": "Mumbai",
            "destination": "Maldives",
            "days": 5,
            "budget": "Luxury",
            "vibe": "Relaxed"
        }
    },
    {
        "name": "Adventure Trek",
        "params": {
            "from_city": "Bangalore",
            "destination": "Manali",
            "days": 6,
            "budget": "Moderate",
            "vibe": "Adventure"
        }
    }
]

print("🎭 Test Scenarios Defined")
print("="*80)
for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n{i}. {scenario['name']}")
    for key, value in scenario['params'].items():
        print(f"   {key}: {value}")

print("\n💡 These scenarios can be tested by changing trip_params and re-running sections 3-9")

## Summary

This notebook demonstrated the complete end-to-end travel planning workflow:

1. ✅ **System Initialization** - LangGraph multi-agent system compiled
2. ✅ **Parameter Setup** - Trip requirements configured
3. ✅ **Agent Execution** - Multi-step workflow executed
4. ✅ **Tool Coordination** - Flight, hotel, weather, and place search tools invoked
5. ✅ **Itinerary Generation** - Complete markdown travel plan created
6. ✅ **Validation** - Output verified for completeness
7. ✅ **Export** - Results saved to file

**Key Achievements:**
- Autonomous agent coordination without manual intervention
- Real API data integration (SerpAPI, OpenWeatherMap, Google Places)
- Structured output following markdown template
- Budget calculations and recommendations
- Execution time tracking and performance metrics

The system is production-ready for deployment via Streamlit or API.